In [1]:
import pandas as pd

from project.thermal import conventional_energy_3uses

Average losses area for envelope components.

In [2]:
ratio_surface = {'Roof': [0.77, 0.29],
                 'Windows': [0.17, 0.19],
                 'Wall': [1.42, 0.78],
                 'Floor': [0.75, 0.28]
                 }

ratio_surface = pd.DataFrame(ratio_surface, index=pd.Index(['Single-family', 'Multi-family'], name='Housing type'))
ratio_surface

,Roof,Windows,Wall,Floor
Housing type,,,,
Single-family,0.77,0.17,1.42,0.75
Multi-family,0.29,0.19,0.78,0.28


# Validation thermal function

This notebook tests the dispersion of space heating consumption and energy performance for best and worst performance building archetypes.

Defining boundaries building archetypes:

In [3]:
# Wall, Roof, Floor, Windows, Air rate (ventilation), Heating system
buildings = {'Worst electricity': [2.5, 2.5, 2, 3, 0.8, 'Electricity-Performance boiler'],
            'Renovation electricity': [0.2, 0.27, 0.12, 1.3, 0.4, 'Electricity-Heat pump'],
            'Renovation efficient electricity': [0.13, 0.2, 0.12, 0.8, 0.4, 'Electricity-Heat pump'],
            'Worst gas': [2.5, 2.5, 2, 3, 0.8, 'Natural gas-Standard boiler'],
            'Renovation gas': [0.2, 0.27, 0.12, 1.3, 0.4, 'Natural gas-Performance boiler'],
            'Renovation efficient gas': [0.13, 0.2, 0.12, 0.8, 0.4, 'Natural gas-Performance boiler'],
            }

replace = {'Electricity-Performance boiler': 0.95,
           'Electricity-Heat pump': 2.5,
           'Natural gas-Standard boiler': 0.6,
          'Natural gas-Performance boiler': 0.76,
           }

buildings = pd.DataFrame(buildings, index=['Wall', 'Roof', 'Floor', 'Windows', 'Ventilation', 'Heating system']).T
buildings['Energy'] = buildings['Heating system'].str.split('-').str[0].rename('Energy')

buildings = pd.concat([buildings, buildings], keys=['Single-family', 'Multi-family'], names=['Housing type'])
#efficiency = pd.Series(envelope.index.get_level_values('Heating system')).replace(replace).set_axis(envelope.index)
efficiency = buildings['Heating system'].replace(replace).set_axis(buildings.index)

buildings = pd.concat((buildings, efficiency.rename('Efficiency')), axis=1)
buildings = buildings.set_index(['Heating system', 'Energy'], append=True)

buildings = buildings.droplevel(None)
buildings = buildings.astype({
    'Wall': 'float',
    'Roof': 'float', 'Floor': 'float', 'Windows': 'float', 'Ventilation': 'float', 'Efficiency': 'float'
})

buildings

Wall  Roof  Floor  \
Housing type  Heating system                 Energy                           
Single-family Electricity-Performance boiler Electricity  2.50  2.50   2.00   
              Electricity-Heat pump          Electricity  0.20  0.27   0.12   
                                             Electricity  0.13  0.20   0.12   
              Natural gas-Standard boiler    Natural gas  2.50  2.50   2.00   
              Natural gas-Performance boiler Natural gas  0.20  0.27   0.12   
                                             Natural gas  0.13  0.20   0.12   
Multi-family  Electricity-Performance boiler Electricity  2.50  2.50   2.00   
              Electricity-Heat pump          Electricity  0.20  0.27   0.12   
                                             Electricity  0.13  0.20   0.12   
              Natural gas-Standard boiler    Natural gas  2.50  2.50   2.00   
              Natural gas-Performance boiler Natural gas  0.20  0.27   0.12   
                                             Natural gas  0.13  0.20   0.12   

                                                          Windows  \
Housing type  Heating system                 Energy                 
Single-family Electricity-Performance boiler Electricity      3.0   
              Electricity-Heat pump          Electricity      1.3   
                                             Electricity      0.8   
              Natural gas-Standard boiler    Natural gas      3.0   
              Natural gas-Performance boiler Natural gas      1.3   
                                             Natural gas      0.8   
Multi-family  Electricity-Performance boiler Electricity      3.0   
              Electricity-Heat pump          Electricity      1.3   
                                             Electricity      0.8   
              Natural gas-Standard boiler    Natural gas      3.0   
              Natural gas-Performance boiler Natural gas      1.3   
                                             Natural gas      0.8   

                                                          Ventilation  \
Housing type  Heating system                 Energy                     
Single-family Electricity-Performance boiler Electricity          0.8   
              Electricity-Heat pump          Electricity          0.4   
                                             Electricity          0.4   
              Natural gas-Standard boiler    Natural gas          0.8   
              Natural gas-Performance boiler Natural gas          0.4   
                                             Natural gas          0.4   
Multi-family  Electricity-Performance boiler Electricity          0.8   
              Electricity-Heat pump          Electricity          0.4   
                                             Electricity          0.4   
              Natural gas-Standard boiler    Natural gas          0.8   
              Natural gas-Performance boiler Natural gas          0.4   
                                             Natural gas          0.4   

                                                          Efficiency  
Housing type  Heating system                 Energy                   
Single-family Electricity-Performance boiler Electricity        0.95  
              Electricity-Heat pump          Electricity        2.50  
                                             Electricity        2.50  
              Natural gas-Standard boiler    Natural gas        0.60  
              Natural gas-Performance boiler Natural gas        0.76  
                                             Natural gas        0.76  
Multi-family  Electricity-Performance boiler Electricity        0.95  
              Electricity-Heat pump          Electricity        2.50  
                                             Electricity        2.50  
              Natural gas-Standard boiler    Natural gas        0.60  
              Natural gas-Performance boiler Natural gas        0.76  
                                      

#### Calculating energy dispersion without ventilation

In [4]:
dpe, energy_primary = conventional_energy_3uses(buildings['Wall'], buildings['Floor'], buildings['Roof'], buildings['Windows'], ratio_surface, buildings['Efficiency'], buildings.index)
result = pd.concat((buildings, dpe, energy_primary), axis=1).sort_index()
result

Wall  Roof  Floor  \
Housing type  Heating system                 Energy                           
Multi-family  Electricity-Heat pump          Electricity  0.20  0.27   0.12   
                                             Electricity  0.13  0.20   0.12   
              Electricity-Performance boiler Electricity  2.50  2.50   2.00   
              Natural gas-Performance boiler Natural gas  0.20  0.27   0.12   
                                             Natural gas  0.13  0.20   0.12   
              Natural gas-Standard boiler    Natural gas  2.50  2.50   2.00   
Single-family Electricity-Heat pump          Electricity  0.20  0.27   0.12   
                                             Electricity  0.13  0.20   0.12   
              Electricity-Performance boiler Electricity  2.50  2.50   2.00   
              Natural gas-Performance boiler Natural gas  0.20  0.27   0.12   
                                             Natural gas  0.13  0.20   0.12   
              Natural gas-Standard boiler    Natural gas  2.50  2.50   2.00   

                                                          Windows  \
Housing type  Heating system                 Energy                 
Multi-family  Electricity-Heat pump          Electricity      1.3   
                                             Electricity      0.8   
              Electricity-Performance boiler Electricity      3.0   
              Natural gas-Performance boiler Natural gas      1.3   
                                             Natural gas      0.8   
              Natural gas-Standard boiler    Natural gas      3.0   
Single-family Electricity-Heat pump          Electricity      1.3   
                                             Electricity      0.8   
              Electricity-Performance boiler Electricity      3.0   
              Natural gas-Performance boiler Natural gas      1.3   
                                             Natural gas      0.8   
              Natural gas-Standard boiler    Natural gas      3.0   

                                                          Ventilation  \
Housing type  Heating system                 Energy                     
Multi-family  Electricity-Heat pump          Electricity          0.4   
                                             Electricity          0.4   
              Electricity-Performance boiler Electricity          0.8   
              Natural gas-Performance boiler Natural gas          0.4   
                                             Natural gas          0.4   
              Natural gas-Standard boiler    Natural gas          0.8   
Single-family Electricity-Heat pump          Electricity          0.4   
                                             Electricity          0.4   
              Electricity-Performance boiler Electricity          0.8   
              Natural gas-Performance boiler Natural gas          0.4   
                                             Natural gas          0.4   
              Natural gas-Standard boiler    Natural gas          0.8   

                                                          Efficiency  0  \
Housing type  Heating system                 Energy                       
Multi-family  Electricity-Heat pump          Electricity        2.50  A   
                                             Electricity        2.50  A   
              Electricity-Performance boiler Electricity        0.95  G   
              Natural gas-Performance boiler Natural gas        0.76  B   
                                             Natural gas        0.76  B   
              Natural gas-Standard boiler    Natural gas        0.60  E   
Single-family Electricity-Heat pump          Electricity        2.50  B   
                                             Electricity        2.50  A   
              Electricity-Performance boiler Electricity        0.95  G   
              Natural gas-Performance boiler Natural gas        0.76  B   
                                             Natural gas     

- The performance of the building archetypes ranges from A to G, which is the necessary range to address the policy in France
- The worst performing fossil fuel-fired multifamily buildings are not G. This is explained by using the average loss surface for each component. In a building, the first and last floors would get more and the other floors less. Here archetypes represent an average apartment in the building. This assumption centers the distribution around the mean.

#### Calculating energy dispersion with ventilation

In [5]:
dpe, energy_primary = conventional_energy_3uses(buildings['Wall'], buildings['Floor'], buildings['Roof'], buildings['Windows'], ratio_surface, buildings['Efficiency'], buildings.index, air_rate=buildings['Ventilation'])
result = pd.concat((buildings, dpe, energy_primary), axis=1).sort_index()
result

Wall  Roof  Floor  \
Housing type  Heating system                 Energy                           
Multi-family  Electricity-Heat pump          Electricity  0.20  0.27   0.12   
                                             Electricity  0.13  0.20   0.12   
              Electricity-Performance boiler Electricity  2.50  2.50   2.00   
              Natural gas-Performance boiler Natural gas  0.20  0.27   0.12   
                                             Natural gas  0.13  0.20   0.12   
              Natural gas-Standard boiler    Natural gas  2.50  2.50   2.00   
Single-family Electricity-Heat pump          Electricity  0.20  0.27   0.12   
                                             Electricity  0.13  0.20   0.12   
              Electricity-Performance boiler Electricity  2.50  2.50   2.00   
              Natural gas-Performance boiler Natural gas  0.20  0.27   0.12   
                                             Natural gas  0.13  0.20   0.12   
              Natural gas-Standard boiler    Natural gas  2.50  2.50   2.00   

                                                          Windows  \
Housing type  Heating system                 Energy                 
Multi-family  Electricity-Heat pump          Electricity      1.3   
                                             Electricity      0.8   
              Electricity-Performance boiler Electricity      3.0   
              Natural gas-Performance boiler Natural gas      1.3   
                                             Natural gas      0.8   
              Natural gas-Standard boiler    Natural gas      3.0   
Single-family Electricity-Heat pump          Electricity      1.3   
                                             Electricity      0.8   
              Electricity-Performance boiler Electricity      3.0   
              Natural gas-Performance boiler Natural gas      1.3   
                                             Natural gas      0.8   
              Natural gas-Standard boiler    Natural gas      3.0   

                                                          Ventilation  \
Housing type  Heating system                 Energy                     
Multi-family  Electricity-Heat pump          Electricity          0.4   
                                             Electricity          0.4   
              Electricity-Performance boiler Electricity          0.8   
              Natural gas-Performance boiler Natural gas          0.4   
                                             Natural gas          0.4   
              Natural gas-Standard boiler    Natural gas          0.8   
Single-family Electricity-Heat pump          Electricity          0.4   
                                             Electricity          0.4   
              Electricity-Performance boiler Electricity          0.8   
              Natural gas-Performance boiler Natural gas          0.4   
                                             Natural gas          0.4   
              Natural gas-Standard boiler    Natural gas          0.8   

                                                          Efficiency  0  \
Housing type  Heating system                 Energy                       
Multi-family  Electricity-Heat pump          Electricity        2.50  A   
                                             Electricity        2.50  A   
              Electricity-Performance boiler Electricity        0.95  G   
              Natural gas-Performance boiler Natural gas        0.76  B   
                                             Natural gas        0.76  B   
              Natural gas-Standard boiler    Natural gas        0.60  E   
Single-family Electricity-Heat pump          Electricity        2.50  A   
                                             Electricity        2.50  A   
              Electricity-Performance boiler Electricity        0.95  G   
              Natural gas-Performance boiler Natural gas        0.76  B   
                                             Natural gas     

Adding ventilation does not change the dispersion

#### Testing rounded value

In [8]:
buildings_round = buildings.copy()
buildings_round['Wall'] = buildings_round['Wall'].round(1)
buildings_round['Floor'] = buildings_round['Floor'].round(1)
buildings_round['Roof'] = buildings_round['Roof'].round(1)
buildings_round['Windows'] = buildings_round['Windows'].round(1)
buildings_round['Efficiency'] = buildings_round['Efficiency'].round(1)

dpe, energy_primary = conventional_energy_3uses(buildings_round['Wall'], buildings_round['Floor'], buildings_round['Roof'], buildings_round['Windows'], ratio_surface, buildings_round['Efficiency'], buildings_round.index, air_rate=buildings['Ventilation'])
result = pd.concat((buildings_round, dpe, energy_primary), axis=1).sort_index()
result

Wall  Roof  Floor  \
Housing type  Heating system                 Energy                           
Multi-family  Electricity-Heat pump          Electricity   0.2   0.3    0.1   
                                             Electricity   0.1   0.2    0.1   
              Electricity-Performance boiler Electricity   2.5   2.5    2.0   
              Natural gas-Performance boiler Natural gas   0.2   0.3    0.1   
                                             Natural gas   0.1   0.2    0.1   
              Natural gas-Standard boiler    Natural gas   2.5   2.5    2.0   
Single-family Electricity-Heat pump          Electricity   0.2   0.3    0.1   
                                             Electricity   0.1   0.2    0.1   
              Electricity-Performance boiler Electricity   2.5   2.5    2.0   
              Natural gas-Performance boiler Natural gas   0.2   0.3    0.1   
                                             Natural gas   0.1   0.2    0.1   
              Natural gas-Standard boiler    Natural gas   2.5   2.5    2.0   

                                                          Windows  \
Housing type  Heating system                 Energy                 
Multi-family  Electricity-Heat pump          Electricity      1.3   
                                             Electricity      0.8   
              Electricity-Performance boiler Electricity      3.0   
              Natural gas-Performance boiler Natural gas      1.3   
                                             Natural gas      0.8   
              Natural gas-Standard boiler    Natural gas      3.0   
Single-family Electricity-Heat pump          Electricity      1.3   
                                             Electricity      0.8   
              Electricity-Performance boiler Electricity      3.0   
              Natural gas-Performance boiler Natural gas      1.3   
                                             Natural gas      0.8   
              Natural gas-Standard boiler    Natural gas      3.0   

                                                          Ventilation  \
Housing type  Heating system                 Energy                     
Multi-family  Electricity-Heat pump          Electricity          0.4   
                                             Electricity          0.4   
              Electricity-Performance boiler Electricity          0.8   
              Natural gas-Performance boiler Natural gas          0.4   
                                             Natural gas          0.4   
              Natural gas-Standard boiler    Natural gas          0.8   
Single-family Electricity-Heat pump          Electricity          0.4   
                                             Electricity          0.4   
              Electricity-Performance boiler Electricity          0.8   
              Natural gas-Performance boiler Natural gas          0.4   
                                             Natural gas          0.4   
              Natural gas-Standard boiler    Natural gas          0.8   

                                                          Efficiency  0  \
Housing type  Heating system                 Energy                       
Multi-family  Electricity-Heat pump          Electricity         2.5  A   
                                             Electricity         2.5  A   
              Electricity-Performance boiler Electricity         1.0  G   
              Natural gas-Performance boiler Natural gas         0.8  B   
                                             Natural gas         0.8  A   
              Natural gas-Standard boiler    Natural gas         0.6  E   
Single-family Electricity-Heat pump          Electricity         2.5  A   
                                             Electricity         2.5  A   
              Electricity-Performance boiler Electricity         1.0  G   
              Natural gas-Performance boiler Natural gas         0.8  B   
                                             Natural gas     

In [7]:
buildings_round['Wall'].astype('float').round(1)

Housing type   Heating system                  Energy     
Single-family  Electricity-Performance boiler  Electricity    2.5
               Electricity-Heat pump           Electricity    0.2
                                               Electricity    0.1
               Natural gas-Standard boiler     Natural gas    2.5
               Natural gas-Performance boiler  Natural gas    0.2
                                               Natural gas    0.1
Multi-family   Electricity-Performance boiler  Electricity    2.5
               Electricity-Heat pump           Electricity    0.2
                                               Electricity    0.1
               Natural gas-Standard boiler     Natural gas    2.5
               Natural gas-Performance boiler  Natural gas    0.2
                                               Natural gas    0.1
Name: Wall, dtype: float64